## Extract, Transform, Load (ETL) Notebook

### Introducción
Este notebook está diseñado para realizar un proceso ETL (Extract, Transform, Load) de forma genérica. 
Exploraremos cada uno de los pasos esenciales con ejemplos de código y explicaciones para guiarte.


In [ ]:
# Bibliotecas necesarias
import pandas as pd
import numpy as np
import sqlalchemy
import json


### 1. Extracción de datos (Extract)
En esta fase, se extraen datos de diversas fuentes como bases de datos, APIs, o archivos locales.
Es importante considerar:
- La estructura de los datos
- Tipos de archivo soportados
- Conexión a bases de datos (usuario, contraseña, host, etc.)
- Manejo de errores en la extracción


In [ ]:
# Ejemplo de extracción de datos desde un archivo CSV
def extract_csv(file_path):
    """
    Extrae datos de un archivo CSV y los devuelve como un DataFrame.
    
    :param file_path: Ruta al archivo CSV.
    :return: DataFrame con los datos extraídos.
    """
    try:
        df = pd.read_csv(file_path)
        print("Datos extraídos exitosamente del archivo CSV.")
        return df
    except Exception as e:
        print(f"Error durante la extracción: {e}")
        return None


### 2. Transformación de datos (Transform)
La transformación es crucial para limpiar y modificar los datos según las necesidades del sistema.
Aspectos clave:
- Limpieza de datos (valores nulos, duplicados, etc.)
- Conversión de formatos
- Creación de nuevas columnas derivadas
- Enriquecimiento con datos externos


In [ ]:
# Ejemplo de transformación de un DataFrame
def transform_data(df):
    """
    Realiza transformaciones en el DataFrame proporcionado.
    
    :param df: DataFrame a transformar.
    :return: DataFrame transformado.
    """
    try:
        # Eliminar valores nulos
        df = df.dropna()
        # Eliminar duplicados
        df = df.drop_duplicates()
        # Crear una nueva columna derivada (ejemplo: suma de dos columnas)
        if 'columna1' in df.columns and 'columna2' in df.columns:
            df['nueva_columna'] = df['columna1'] + df['columna2']
        print("Transformación de datos completada.")
        return df
    except Exception as e:
        print(f"Error durante la transformación: {e}")
        return None


### 3. Carga de datos (Load)
Esta fase implica cargar los datos transformados a un destino como una base de datos, un archivo, o una API.
Consideraciones importantes:
- Formato del destino
- Validación de datos cargados
- Eficiencia del proceso


In [ ]:
# Ejemplo de carga de datos a una base de datos usando SQLAlchemy
def load_to_database(df, connection_string, table_name):
    """
    Carga un DataFrame en una tabla de base de datos.
    
    :param df: DataFrame a cargar.
    :param connection_string: Cadena de conexión a la base de datos.
    :param table_name: Nombre de la tabla de destino.
    """
    try:
        engine = sqlalchemy.create_engine(connection_string)
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Datos cargados exitosamente en la tabla {table_name}.")
    except Exception as e:
        print(f"Error durante la carga: {e}")


### Ejecución del proceso ETL


In [ ]:
# Ruta al archivo de entrada (modificar según sea necesario)
file_path = "datos.csv"
connection_string = "sqlite:///mi_base_de_datos.db"
table_name = "tabla_final"

# Pasos del proceso ETL
print("\n--- Inicio del Proceso ETL ---\n")

# Extracción
datos = extract_csv(file_path)
if datos is not None:
    
    # Transformación
    datos_transformados = transform_data(datos)
    
    if datos_transformados is not None:
        
        # Carga
        load_to_database(datos_transformados, connection_string, table_name)

print("\n--- Fin del Proceso ETL ---\n")

### CUADRO DE PRUEBA
subida desde VS Code